In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract data science jobs csv into DataFrame

In [2]:
# Read in the data science jobs csv
datasci_file = "../Resources/datascientist_jobs_in_australia_Oct_25_2019.csv"
datasci_df = pd.read_csv(datasci_file)
datasci_df.head()

,Unnamed: 0,title,company,cpage,ratings,location,days_ago,summary
0,1,Deloitte Access Economics - Junior Data Scientist,Deloitte,https://au.indeed.com/cmp/Deloitte,4.0,Sydney NSW,30,The data will be both temporal and spatial. Ad...
1,2,Data Scientist,Australian Government Department of Human Serv...,https://au.indeed.com/cmp/Australian-Governmen...,3.9,Canberra ACT,10,They lead the implementation of data science a...
2,3,Junior Data Analyst/Scientist,International Institute of Data & Analytics,https://au.indeed.comn/a,0.0,Sydney NSW,10,"In data science and big data analytics, the ID..."
3,4,IBM Research Scientist Data Scientist – Melbourne,IBM,https://au.indeed.com/cmp/IBM,3.9,Melbourne VIC,30,"Experience working with big data sets, especia..."
4,5,Data Scientist,Cash Converters,https://au.indeed.com/cmp/Cash-Converters,3.5,Taringa QLD,1,Collaborates with data/software engineers to i...


In [3]:
# Summary of DataFrame
datasci_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619 entries, 0 to 618
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  619 non-null    int64  
 1   title       619 non-null    object 
 2   company     619 non-null    object 
 3   cpage       619 non-null    object 
 4   ratings     619 non-null    float64
 5   location    619 non-null    object 
 6   days_ago    619 non-null    int64  
 7   summary     619 non-null    object 
dtypes: float64(1), int64(2), object(5)
memory usage: 38.8+ KB


### Transform data science jobs csv

In [4]:
# Rename the unnamed column to id and set as index
datasci_df.rename(columns={"Unnamed: 0":"id"},inplace=True)
datasci_df.set_index("id",inplace=True)

In [5]:
# Preview resulting DataFrame
datasci_df.head()

,title,company,cpage,ratings,location,days_ago,summary
id,,,,,,,
1,Deloitte Access Economics - Junior Data Scientist,Deloitte,https://au.indeed.com/cmp/Deloitte,4.0,Sydney NSW,30,The data will be both temporal and spatial. Ad...
2,Data Scientist,Australian Government Department of Human Serv...,https://au.indeed.com/cmp/Australian-Governmen...,3.9,Canberra ACT,10,They lead the implementation of data science a...
3,Junior Data Analyst/Scientist,International Institute of Data & Analytics,https://au.indeed.comn/a,0.0,Sydney NSW,10,"In data science and big data analytics, the ID..."
4,IBM Research Scientist Data Scientist – Melbourne,IBM,https://au.indeed.com/cmp/IBM,3.9,Melbourne VIC,30,"Experience working with big data sets, especia..."
5,Data Scientist,Cash Converters,https://au.indeed.com/cmp/Cash-Converters,3.5,Taringa QLD,1,Collaborates with data/software engineers to i...


In [6]:
# Check for duplicate rows, based on all columns
duplicate_rows = datasci_df[datasci_df.duplicated(keep="first")]
# duplicate_rows
# duplicate_rows.count()

In [7]:
# Drop duplicate rows from DataFrame
datasci_df.drop_duplicates(inplace=True)

In [8]:
# Clean up cpage column (replace invalid URL 'https://au.indeed.comn/a' with 'Not available')
datasci_df.replace("https://au.indeed.comn/a","Not available",inplace=True)

In [9]:
# Replace 0.0 ratings with 'No rating'
datasci_df.replace(0.00,"Not rating",inplace=True)

In [10]:
# Drop days_ago columns
datasci_df.drop(columns="days_ago",inplace=True)

In [11]:
# Replace state only values in location with "N/A, <state>" to enable splitting of location into city and state
datasci_df["location"].replace(to_replace="New South Wales",value="Not available, NSW",inplace=True)
datasci_df["location"].replace(to_replace="Victoria",value="Not available, VIC",inplace=True)
datasci_df["location"].replace(to_replace="Queensland",value="Not available, QLD",inplace=True)
datasci_df["location"].replace(to_replace="Tasmania",value="Not available, TAS",inplace=True)

In [12]:
# Clean up location column; separate city and state into two columns
location = datasci_df["location"]
# location.unique()
# location.value_counts()
location_df = location.str.rsplit(" ",n=1,expand=True)

In [13]:
# Add new city and state columns into existing DataFrame
datasci_df["city"] = location_df[0]
datasci_df["state"] = location_df[1]
# Drop existing location column
datasci_df.drop(columns="location",inplace=True)

In [14]:
# Check city and state data
# datasci_df["city"].unique()
# datasci_df["state"].unique()

In [15]:
# Clean up city column to replace "Australia" values? OR leave "Australia" amd rename column to location
# datasci_df[datasci_df["state"].isna()]
# datasci_df.loc[datasci_df["state"] == "Australia"]

In [16]:
# Rename columns with meaningful titles e.g. 'Indeed landing page'
datasci_df.rename(columns={"title": "job posting title",
                            "company": "organisation",
                            "cpage": "Indeed profile page",
                            "ratings": "Indeed rating",
                            "summary": "job summary",
                            "city": "location"},
                            inplace=True)

In [17]:
# Select columns from data sci DF to create a job information table
jobinfo_cols = ["job posting title","organisation","location","state"]
jobinfo_df = datasci_df[jobinfo_cols].copy()
jobinfo_df.head()

,job posting title,organisation,location,state
id,,,,
1,Deloitte Access Economics - Junior Data Scientist,Deloitte,Sydney,NSW
2,Data Scientist,Australian Government Department of Human Serv...,Canberra,ACT
3,Junior Data Analyst/Scientist,International Institute of Data & Analytics,Sydney,NSW
4,IBM Research Scientist Data Scientist – Melbourne,IBM,Melbourne,VIC
5,Data Scientist,Cash Converters,Taringa,QLD


In [18]:
# Select columns to create organisation DF
orginfo_cols = ["organisation","Indeed profile page","Indeed rating"]
orginfo_df = datasci_df[orginfo_cols].copy()
orginfo_df

,organisation,Indeed profile page,Indeed rating
id,,,
1,Deloitte,https://au.indeed.com/cmp/Deloitte,4
2,Australian Government Department of Human Serv...,https://au.indeed.com/cmp/Australian-Governmen...,3.9
3,International Institute of Data & Analytics,Not available,Not rating
4,IBM,https://au.indeed.com/cmp/IBM,3.9
5,Cash Converters,https://au.indeed.com/cmp/Cash-Converters,3.5
...,...,...,...
564,Hudson,https://au.indeed.com/cmp/Hudson,3.6
578,Real Time Australia,Not available,Not rating
579,Real Time Australia,Not available,Not rating


In [19]:
# Drop duplicates from org DF
orginfo_df.drop_duplicates(subset="organisation",inplace=True)

In [20]:
# Final, deduped org into DF
orginfo_df

,organisation,Indeed profile page,Indeed rating
id,,,
1,Deloitte,https://au.indeed.com/cmp/Deloitte,4
2,Australian Government Department of Human Serv...,https://au.indeed.com/cmp/Australian-Governmen...,3.9
3,International Institute of Data & Analytics,Not available,Not rating
4,IBM,https://au.indeed.com/cmp/IBM,3.9
5,Cash Converters,https://au.indeed.com/cmp/Cash-Converters,3.5
...,...,...,...
517,Langley Chase,Not available,Not rating
518,Beacham Group Pty Ltd,Not available,Not rating
522,Talentpool Recruitment,Not available,Not rating


### Extract ABS labour market data csv into DataFrame



In [21]:
# Read in csv
abslabour_file = "../Resources/SA4 Time Series - July 2020.csv"
abslabour_df = pd.read_csv(abslabour_file)
abslabour_df.head()

,State/Territory,Date,Employment Rate (15-64),Unemployment Rate (15+),Participation Rate (15+)
0,NSW,Feb-78,NaN,6.9,60.8
1,NSW,Mar-78,NaN,6.4,59.8
2,NSW,Apr-78,63.7,6.5,60.2
3,NSW,May-78,63.7,6.1,60.0
4,NSW,Jun-78,63.7,6.4,60.1


### Transform ABS labour market data

In [22]:
# Filter for relevant labour market data dates (Oct 2019 matched jobs data)
abslabour_oct2019_df = abslabour_df.loc[abslabour_df["Date"] == "Oct-19"]
abslabour_oct2019_df

,State/Territory,Date,Employment Rate (15-64),Unemployment Rate (15+),Participation Rate (15+)
500,NSW,Oct-19,74.9,4.8,65.5
1010,VIC,Oct-19,74.0,4.8,66.4
1520,QLD,Oct-19,73.9,6.4,66.0
2030,SA,Oct-19,72.0,6.2,62.7
2540,WA,Oct-19,74.7,5.7,68.2
3050,TAS,Oct-19,71.2,6.0,60.6
3560,NT,Oct-19,74.3,5.6,73.4
4070,ACT,Oct-19,79.1,3.0,70.8
4580,Australia,Oct-19,74.3,5.3,66.0


In [23]:
# Drop row with value "Australia" in state/territory column
abslabour_oct2019_df.drop(4580,inplace=True)

In [24]:
abslabour_oct2019_df.reset_index(drop=True, inplace=True)
abslabour_oct2019_df

,State/Territory,Date,Employment Rate (15-64),Unemployment Rate (15+),Participation Rate (15+)
0,NSW,Oct-19,74.9,4.8,65.5
1,VIC,Oct-19,74.0,4.8,66.4
2,QLD,Oct-19,73.9,6.4,66.0
3,SA,Oct-19,72.0,6.2,62.7
4,WA,Oct-19,74.7,5.7,68.2
5,TAS,Oct-19,71.2,6.0,60.6
6,NT,Oct-19,74.3,5.6,73.4
7,ACT,Oct-19,79.1,3.0,70.8


In [25]:
# Rename columns
abslabour_oct2019_df.columns=["state","date","employment rate", "unemployment rate", "participation rate"]

In [26]:
abslabour_oct2019_df

,state,date,employment rate,unemployment rate,participation rate
0,NSW,Oct-19,74.9,4.8,65.5
1,VIC,Oct-19,74.0,4.8,66.4
2,QLD,Oct-19,73.9,6.4,66.0
3,SA,Oct-19,72.0,6.2,62.7
4,WA,Oct-19,74.7,5.7,68.2
5,TAS,Oct-19,71.2,6.0,60.6
6,NT,Oct-19,74.3,5.6,73.4
7,ACT,Oct-19,79.1,3.0,70.8


In [27]:
# Select columns from abslabour DF to create a labour market information table
labourinfo_cols = ["state","employment rate","unemployment rate","participation rate"]
labourinfo_df = abslabour_oct2019_df[labourinfo_cols].copy()
labourinfo_df

,state,employment rate,unemployment rate,participation rate
0,NSW,74.9,4.8,65.5
1,VIC,74.0,4.8,66.4
2,QLD,73.9,6.4,66.0
3,SA,72.0,6.2,62.7
4,WA,74.7,5.7,68.2
5,TAS,71.2,6.0,60.6
6,NT,74.3,5.6,73.4
7,ACT,79.1,3.0,70.8


### List of final DFs for database tables

In [28]:
# Job information
jobinfo_df

,job posting title,organisation,location,state
id,,,,
1,Deloitte Access Economics - Junior Data Scientist,Deloitte,Sydney,NSW
2,Data Scientist,Australian Government Department of Human Serv...,Canberra,ACT
3,Junior Data Analyst/Scientist,International Institute of Data & Analytics,Sydney,NSW
4,IBM Research Scientist Data Scientist – Melbourne,IBM,Melbourne,VIC
5,Data Scientist,Cash Converters,Taringa,QLD
...,...,...,...,...
564,Senior Research Analyst,Hudson,Melbourne,VIC
578,6M Contract - Automation Engineer - JavaScript...,Real Time Australia,Melbourne,VIC
579,Full Stack Senior Software Engineer (React.js ...,Real Time Australia,Melbourne,VIC


In [29]:
# Organisation information
orginfo_df

,organisation,Indeed profile page,Indeed rating
id,,,
1,Deloitte,https://au.indeed.com/cmp/Deloitte,4
2,Australian Government Department of Human Serv...,https://au.indeed.com/cmp/Australian-Governmen...,3.9
3,International Institute of Data & Analytics,Not available,Not rating
4,IBM,https://au.indeed.com/cmp/IBM,3.9
5,Cash Converters,https://au.indeed.com/cmp/Cash-Converters,3.5
...,...,...,...
517,Langley Chase,Not available,Not rating
518,Beacham Group Pty Ltd,Not available,Not rating
522,Talentpool Recruitment,Not available,Not rating


In [30]:
# Labour market information
labourinfo_df

,state,employment rate,unemployment rate,participation rate
0,NSW,74.9,4.8,65.5
1,VIC,74.0,4.8,66.4
2,QLD,73.9,6.4,66.0
3,SA,72.0,6.2,62.7
4,WA,74.7,5.7,68.2
5,TAS,71.2,6.0,60.6
6,NT,74.3,5.6,73.4
7,ACT,79.1,3.0,70.8
